In [14]:
%%bash
python3 --version

Python 3.12.8


In [15]:
import pyspark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import Row, SQLContext
from pyspark.sql.functions import broadcast
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F

from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    DoubleType,
    DateType,
)
import random
from datetime import datetime, timedelta

pyspark.__version__

'3.5.3'

In [16]:
spark = (
    SparkSession.builder.appName("testing optimization")
    .master("spark://localhost:7077")
    # .config("spark.sql.execution.arrow.pyspark.selfDestruct.enabled", "true")
    # .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .getOrCreate()
)
spark

In [17]:
# Define schema
schema = StructType(
    [
        StructField("user_id", StringType(), False),
        StructField("age", IntegerType(), True),
        StructField("purchase_amount", DoubleType(), True),
        StructField("transaction_date", DateType(), True),
        StructField("product_category", StringType(), True),
    ]
)


# Helper functions to generate random data
def generate_user_id():
    return "121asd"


def generate_age():
    return random.randint(18, 80)


def generate_amount():
    return round(random.uniform(10.0, 1000.0), 2)


def generate_date():
    start_date = datetime(2023, 1, 1)
    days = random.randint(0, 365)
    return (start_date + timedelta(days=days)).date()


def generate_category():
    categories = ["Electronics", "Clothing", "Books", "Home", "Food", "Sports"]
    return random.choice(categories)


# Generate data
num_rows = 100  # 1 million rows
data = []
for _ in range(num_rows):
    row = (
        generate_user_id(),
        generate_age(),
        generate_amount(),
        generate_date(),
        generate_category(),
    )
    data.append(row)

# Create DataFrame
df = spark.createDataFrame(data, schema)
df.show()

+-------+---+---------------+----------------+----------------+
|user_id|age|purchase_amount|transaction_date|product_category|
+-------+---+---------------+----------------+----------------+
| 121asd| 45|         440.75|      2023-08-01|           Books|
| 121asd| 51|          84.16|      2023-05-05|            Food|
| 121asd| 66|         433.85|      2023-03-31|          Sports|
| 121asd| 53|         808.78|      2023-06-23|        Clothing|
| 121asd| 29|         792.19|      2023-11-15|            Home|
| 121asd| 72|          96.51|      2023-02-10|           Books|
| 121asd| 20|         465.72|      2023-11-04|            Food|
| 121asd| 52|         451.78|      2023-03-22|        Clothing|
| 121asd| 78|         395.95|      2023-03-14|            Home|
| 121asd| 62|          529.7|      2023-04-27|            Food|
| 121asd| 42|         536.43|      2023-09-12|            Home|
| 121asd| 18|         680.06|      2023-12-21|            Food|
| 121asd| 37|         399.32|      2023-

In [22]:
# do the write do the write
df.repartition(1).write.format("csv").mode("overwrite").option("header", "true").save(
    "./tmp/file.csv"
)

24/12/17 10:34:42 WARN TaskSetManager: Lost task 0.0 in stage 14.0 (TID 31) (172.18.0.4 executor 1): java.io.IOException: Mkdirs failed to create file:/Users/lechithanh/workspace/spark-playground/notebook/tmp/file.csv/_temporary/0/_temporary/attempt_20241217103442301290895833071480_0014_m_000000_31 (exists=false, cwd=file:/opt/bitnami/spark/work/app-20241217031916-0000/1)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStream(CodecStreams.scala:81)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStreamWriter(CodecStreams.scala:92)
	at org.apache.spark.sql.execution.datasources.csv.CsvOutp

Py4JJavaError: An error occurred while calling o129.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 4 times, most recent failure: Lost task 0.3 in stage 14.0 (TID 34) (172.18.0.4 executor 1): java.io.IOException: Mkdirs failed to create file:/Users/lechithanh/workspace/spark-playground/notebook/tmp/file.csv/_temporary/0/_temporary/attempt_20241217103442301290895833071480_0014_m_000000_34 (exists=false, cwd=file:/opt/bitnami/spark/work/app-20241217031916-0000/1)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStream(CodecStreams.scala:81)
	at org.apache.spark.

In [21]:
%%bash
chmod 777 tmp

In [1]:
def get_base_mertrics() -> dict:
    """
    get based info from databricks context (dbutils)
    """
    return {}


# --- do sth to get the context of spark structured streaming
def get_based_metrics() -> dict: ...


def from_dict_to_final_model() -> dict: ...

In [2]:
schema = StructType(
    [
        StructField("variable_1", StringType(), True),
        StructField("variable_1", StringType(), True),
    ]
)


sc = spark.sparkContext
rdd = sc.paralellize({"variable_1": "abc", "variable_2": "bcde"})

spark.createDataFrame(schema, rdd)

NameError: name 'StructType' is not defined